In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.156:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1618505751278)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [2]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)
graph.vertices.collect

graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@6075d15c
res0: Array[(org.apache.spark.graphx.VertexId, Boolean)] = Array((34,false), (4,false), (16,false), (22,false), (28,false), (30,false), (14,false), (32,false), (24,false), (6,false), (8,false), (12,false), (18,false), (20,false), (26,false), (10,false), (2,false), (13,false), (19,false), (15,false), (21,false), (25,false), (29,false), (11,false), (27,false), (33,false), (23,false), (1,false), (17,false), (3,false), (7,false), (9,false), (31,false), (5,false))


In [3]:
def independant_cascade(graph:Graph[String,Float]):Graph[String,Float]={
    def mergeMsg (m1:String, m2:String):String = {
        if(m1=="become_inactive" || m2 == "become_inactive"){return "become_inactive"}
        if (m1 == "become_infected" || m2 == "become_infected"){return "become_infected"}
        return "nothing"
    }

    def vprog(VertexId:VertexId, VD:String, A:String): String ={
        if (VD == "susceptible" && A == "become_infected"){ return "infected"}
        if(VD=="infected" && A == "become_inactive"){return "inactive"}
        return VD
    }

    def sendMsg(triplet:EdgeTriplet[String, Float]): Iterator[(VertexId, String)]={
        if (triplet.srcAttr == "infected"){
            if (scala.util.Random.nextFloat < triplet.attr){
                return Iterator((triplet.dstId,"become_infected"),(triplet.srcId,"become_inactive"))
            }
        return Iterator((triplet.srcId,"become_inactive"))
        }
        return Iterator()
    }
    
    return graph.pregel("nothing",Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
    .mapVertices((vid,vdata) => if(vdata == "infected") "inactive" else vdata)
}

independant_cascade: (graph: org.apache.spark.graphx.Graph[String,Float])org.apache.spark.graphx.Graph[String,Float]


In [4]:
independant_cascade(graph).vertices.collect

<console>: 37: error: type mismatch;

In [5]:
type VD = (String,Float,Float)
type ED = Float
type Message = (String,Float)

def linear_threshhold(graph:Graph[VD,ED]):Graph[VD,ED]={
    def mergeMsg (m1:Message, m2:Message):Message = {
        if(m1._1=="become_inactive" || m2._1 == "become_inactive"){return ("become_inactive",0f)}
        if (m1._1 == "become_infected" && m2._1 == "become_infected"){return ("become_infected",m1._2+m2._2)}
        if (m1._1 == "become_infected" ){return m1}
        if (m2._1 == "become_infected" ){return m2}
        return ("nothing",0f)
    }

    def vprog(VertexId:VertexId, vdata:VD, m:Message): VD ={
        if (m._1 == "become_inactive"){
            return ("inactive", vdata._2, vdata._3)
        }
        if (m._1 == "become_infected"){
            val state = vdata._1
            val new_infection_rate = vdata._2 + m._2
            val threshold = vdata._3
            if (state == "susceptible" && new_infection_rate > threshold){
                return ("infected",new_infection_rate,threshold)
            }
            return (state,new_infection_rate,threshold)
        }
        return vdata
    }

    def sendMsg(triplet:EdgeTriplet[VD, ED]): Iterator[(VertexId, Message)]={
        if (triplet.srcAttr._1 == "infected"){
            return Iterator((triplet.dstId,("become_infected",triplet.attr)),(triplet.srcId,("become_inactive",0f)))

        return Iterator((triplet.srcId,("become_inactive",0f)))
        }
        return Iterator()
    }
    
    return graph.pregel(("nothing",0f),Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
}

defined type alias VD
defined type alias ED
defined type alias Message
linear_threshhold: (graph: org.apache.spark.graphx.Graph[VD,ED])org.apache.spark.graphx.Graph[VD,ED]


In [6]:
def normalize_graph_weights[VD](graph:Graph[VD,Float])(implicit m: ClassManifest[VD]):Graph[VD,Float] = {
    val weight_sums_per_dst = graph.triplets.map(t => (t.dstId,t.attr))
                            .reduceByKey( _ + _)
    val newEdges = graph.edges.map(e => (e.dstId,e))
         .join(weight_sums_per_dst).map{ case ((dstId,(edge,weightSum)))=>Edge(edge.srcId,edge.dstId,edge.attr/weightSum)}
    return Graph(graph.vertices,newEdges)
}

val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices[VD]((vid,i) =>  if(scala.util.Random.nextFloat <0.1) ("infected",1f, scala.util.Random.nextFloat) else ("susceptible",0f, scala.util.Random.nextFloat))
graph.edges.collect

normalize_graph_weights: [VD](graph: org.apache.spark.graphx.Graph[VD,Float])(implicit m: ClassManifest[VD])org.apache.spark.graphx.Graph[VD,Float]
graph: org.apache.spark.graphx.Graph[VD,Float] = org.apache.spark.graphx.impl.GraphImpl@36db5af3
res2: Array[org.apache.spark.graphx.Edge[Float]] = Array(Edge(2,1,0.5812977), Edge(3,1,0.6295288), Edge(3,2,0.64329004), Edge(4,1,0.2192089), Edge(4,2,0.6613361), Edge(4,3,0.8056748), Edge(5,1,0.27176243), Edge(6,1,0.77093744), Edge(7,1,0.47805238), Edge(7,5,0.9960699), Edge(7,6,0.65685326), Edge(8,1,0.8495408), Edge(8,2,0.82857066), Edge(8,3,0.62851644), Edge(8,4,0.71172094), Edge(9,1,0.1044271), Edge(9,3,0.30305225), Edge(10,3,0.47695982), Edge(11,1,0.43643355), Edge(11,5,0.9067097), Edge(11,6,0.15488851), Edge(12,1,0.5849967), Edge(13,1,0.4426...


In [7]:
normalize_graph_weights(graph).edges.collect

res3: Array[org.apache.spark.graphx.Edge[Float]] = Array(Edge(3,2,0.14839081), Edge(4,2,0.15255359), Edge(7,6,0.4042399), Edge(8,2,0.19113038), Edge(8,4,0.42709345), Edge(11,6,0.09532131), Edge(13,4,0.3161904), Edge(14,2,0.2113597), Edge(14,4,0.25671613), Edge(17,6,0.5004388), Edge(18,2,0.02638042), Edge(20,2,0.0017327544), Edge(22,2,0.098358065), Edge(26,24,0.054469343), Edge(28,24,0.64549035), Edge(30,24,0.18485333), Edge(31,2,0.17009422), Edge(32,26,1.0), Edge(33,16,0.77688086), Edge(33,24,0.06842391), Edge(33,30,0.930963), Edge(33,32,0.02134499), Edge(34,10,1.0), Edge(34,14,1.0), Edge(34,16,0.22311912), Edge(34,20,1.0), Edge(34,24,0.046763178), Edge(34,28,1.0), Edge(34,30,0.069037005), Edge(34,32,0.978655), Edge(34,34,1.0), Edge(2,1,0.07486691), Edge(3,1,0.08107873), Edge(4,1,0.0282...


In [8]:
linear_threshhold(graph).vertices.filter(vdata => vdata._2._2 >0).collect

res4: Array[(org.apache.spark.graphx.VertexId, VD)] = Array((4,(susceptible,0.5269089,0.55950093)), (16,(infected,1.0,0.916711)), (2,(inactive,0.64329004,0.48665196)), (13,(inactive,1.0,0.5158876)), (23,(infected,1.0,0.9336666)), (1,(infected,1.7578967,0.91711354)), (3,(inactive,1.0,0.10067272)), (9,(inactive,1.0,0.8100585)))


In [9]:
def selectVerticeWithMaxDegree(graph:Graph[Boolean,Float]):Graph[Boolean,Float] = {
    val maxDegreeVertice = graph.triplets.filter(triplet => !triplet.srcAttr && !triplet.dstAttr)
                             .map(triplet => (triplet.srcId,1))
                             .reduceByKey(_+_)
                             .reduce((v1,v2) => if(v1._2 >= v2._2 ) v1 else v2)._1
    return graph.mapVertices((vid, vdata) => vid == maxDegreeVertice || vdata)
}

val selectKVerticesWithMaxDegrees = (k:Int) => Function.chain(List.fill(k)(selectVerticeWithMaxDegree _ ))

selectVerticeWithMaxDegree: (graph: org.apache.spark.graphx.Graph[Boolean,Float])org.apache.spark.graphx.Graph[Boolean,Float]
selectKVerticesWithMaxDegrees: Int => (org.apache.spark.graphx.Graph[Boolean,Float] => org.apache.spark.graphx.Graph[Boolean,Float]) = $Lambda$3299/0x00000008411b5040@67c4c95e


In [93]:
def selectKVerticeRandom (k:Float,graph:Graph[Boolean,Float]):Graph[Boolean,Float] = {
    val selected = graph.vertices.sample(false,10f/graph.vertices.count())
    return graph.outerJoinVertices(selected) { (id, oldAttr, value) =>
              value match {
                case Some(_) => true
                case None => false // None means the node was not selected
              }
        }
}

selectKVerticeRandom: (k: Float, graph: org.apache.spark.graphx.Graph[Boolean,Float])org.apache.spark.graphx.Graph[Boolean,Float]


In [14]:
selectKVerticesWithMaxDegrees (10) (graph).vertices.join(graph.outDegrees).collect

res7: Array[(org.apache.spark.graphx.VertexId, (Boolean, Int))] = Array((34,(true,18)), (4,(true,3)), (22,(true,2)), (28,(true,3)), (30,(false,2)), (14,(true,4)), (32,(true,4)), (6,(false,1)), (8,(true,4)), (12,(false,1)), (18,(false,2)), (20,(false,2)), (26,(false,2)), (10,(false,1)), (2,(false,1)), (13,(false,2)), (29,(false,1)), (11,(true,3)), (33,(true,11)), (17,(false,2)), (3,(false,2)), (7,(true,3)), (9,(false,2)), (31,(false,2)), (5,(false,1)))


In [15]:
graph.mapVertices((vid,selected) =>  if (selected) "infected" else "susceptible")

res8: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@39fa0c19


In [16]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)

val graph_init = selectKVerticesWithMaxDegrees (10) (graph).mapVertices(
                    (vid,selected) =>  if (selected) "infected" else "susceptible")

val graph_result = independant_cascade(graph_init)

graph_result.vertices.map(v => if (v._2 == "susceptible") 0 else 1).reduce((a,b) => a+b)

graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@12fb28fa
graph_init: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@59392e06
graph_result: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@4fcdacbf
res9: Int = 20


In [17]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)

val graph_init = selectKVerticesWithMaxDegrees (10) (graph).mapVertices(
                    (vid,selected) =>  if (selected) "infected" else "susceptible")

val graph_result = independant_cascade(graph_init)

graph_result.vertices.map(v => if (v._2 == "susceptible") 0 else 1).reduce((a,b) => a+b)

graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@7bfb7662
graph_init: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@41be2133
graph_result: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@78f8f39c
res10: Int = 24


In [105]:
val edges =  sc.textFile("data/soc-sign-bitcoinotc.csv")
                .map(f => f.split(","))
                .map(a => Edge(a(0).toLong, a(1).toLong, a(2).toLong / 10f))
                .filter(e => (e.attr > 0))

val vertices = edges.map(e => (e.srcId, false)).distinct()

val graph = Graph(vertices,edges)

val graph_init = selectKVerticesWithMaxDegrees (10) (graph).mapVertices(
                    (vid,selected) =>  if (selected) "infected" else "susceptible")

val graph_result = independant_cascade(graph_init)

graph_result.vertices.map(v => if (v._2 == "susceptible") 0 else 1).reduce((a,b) => a+b)

edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Float]] = MapPartitionsRDD[9240] at filter at <console>:47
vertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Boolean)] = MapPartitionsRDD[9244] at distinct at <console>:49
graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@1f3af1c0
graph_init: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@3c45109d
graph_result: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@49e9eb5c
res58: Int = 1801


In [112]:
val edges =  sc.textFile("data/soc-sign-bitcoinotc.csv")
                .map(f => f.split(","))
                .map(a => Edge(a(0).toLong, a(1).toLong, a(2).toLong / 10f))
                .filter(e => (e.attr > 0))

val vertices = edges.map(e => (e.srcId, false)).distinct()

val graph = Graph(vertices,edges)

val graph_init = selectKVerticeRandom (10,graph).mapVertices(
                    (vid,selected) =>  if (selected) "infected" else "susceptible")

val graph_result = independant_cascade(graph_init)

graph_result.vertices.map(v => if (v._2 == "susceptible") 0 else 1).reduce((a,b) => a+b)

edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Float]] = MapPartitionsRDD[10744] at filter at <console>:47
vertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Boolean)] = MapPartitionsRDD[10748] at distinct at <console>:49
graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@14d3f003
graph_init: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@fd562bb
graph_result: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@6678748c
res65: Int = 1813
